## PCX - Phuture Crypto Market Index - Methodology Template

## Configuring packages

In [4]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [5]:
min_mcap =  5e8
min_weight = 0.005
max_weight = 0.2
circ_supply_threshold = 0.3
liveness_threshold = 365 # In days
liquidity_consistency = 90 # In days
coingecko_category = None

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')
liquid_staking_tokens = pd.DataFrame(cg.get_coins_markets('usd',category='liquid-staking-tokens')).set_index('id')

methodology_class = MethodologyBase(min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [6]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens,liquid_staking_tokens],
    ids_to_replace=[['bitcoin'],['wrapped-bitcoin']],
    remove_category_assets=['wrapped-bitcoin'],
    values_to_update=[{'id':'matic-network','category':'platforms','value':{}}]
    )
liquidity

Excluding mantle, pricing data available only for 180 < 365 days
Excluding beam-2, pricing data available only for 79 < 365 days
Excluding blur, pricing data available only for 333 < 365 days
Excluding pepe, pricing data available only for 270 < 365 days
Excluding corgiai, pricing data available only for 186 < 365 days
0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
0x925206b8a707096ed26ae47c84747fe0bb734f59 ethereum
0x19de6b897ed14a376dda0fe53a5420d2ac828a28 ethereum
0x7205705771547cf79201111b4bd8aaf29467b9ec polygon-pos
 ethereum
0x44c784266cf024a60e8acf2427b9857ace194c5d avalanche
0x23ee2343b892b1bb63503a4fabc840e0e2c6810f optimistic-ethereum
0x23ee2343b892b1bb63503a4fabc840e0e2c6810f base
0x8b1f4432f943c465a973fedc6d7aa50fc96f1f65 fantom
0xa0b862f60edef4452f25b4160f177db44deb6cf1 arbitrum-one
0x7c598c96d02398d89fbcb9d41eab3df0c16f227d fantom


,spot price,delivery price,slippage,blockchain
id,,,,
wrapped-bitcoin,0.000022,0.000022,0.000000,ethereum
woo-network,2.223535,2.222914,-0.000279,ethereum
ethereum,0.000373,0.000373,-0.000385,ethereum
avalanche-2,0.026022,0.026006,-0.000601,avalanche
binancecoin,0.003209,0.003206,-0.001032,binance-smart-chain
pancakeswap-token,0.325068,0.324376,-0.002130,binance-smart-chain
matic-network,1.063138,1.060649,-0.002342,polygon-pos
illuvium,0.010314,0.010288,-0.002497,ethereum
uniswap,0.145450,0.145056,-0.002712,arbitrum-one


## Show Results

In [7]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
wrapped-bitcoin,BTC,896558259784,0.210959,13826,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,ethereum
ethereum,ETH,320500809740,0.210959,13826,0x0000000000000000000000000000000000000000,ethereum
binancecoin,BNB,47966274120,0.210959,13826,0x0000000000000000000000000000000000000000,binance-smart-chain
avalanche-2,AVAX,14038903013,0.072911,4779,0x0000000000000000000000000000000000000000,avalanche
matic-network,MATIC,8666966291,0.045012,2950,0x0000000000000000000000000000000000000000,polygon-pos
chainlink,LINK,8589434717,0.044609,2924,0x514910771af9ca656af840dff83e8264ecf986ca,ethereum
the-open-network,TON,7474680506,0.038820,2545,0x582d872a1b094fc48f5de31d3b73f2d9be47def1,ethereum
shiba-inu,SHIB,6016763206,0.031248,2048,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,ethereum
uniswap,UNI,5156352793,0.026780,1755,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
